# Téléchargement du fichier cif

In [ ]:
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.ext.matproj import MPRester
from pymatgen.io.cif import CifWriter
from IPython.display import FileLink
from pymatgen.electronic_structure.plotter import BSPlotter
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
key = "xFkmtLT8p034lKKT"
material = "mp-961682"
m = MPRester(key)
s = m.get_structure_by_material_id(material,conventional_unit_cell=True)
doc = CifWriter(s)
doc.write_file(material+".cif")
display(FileLink(material+".cif"))

# Détermination bande interdite

In [ ]:
bandstruct = m.get_bandstructure_by_material_id(material)
bsplotter = BSPlotter(bandstruct)
bsplotter.show()

print("Informations de la bande interdite : ",bandstruct.get_band_gap())



#bien dire qu'on est en semi conducteur : bande interdite indirecte et de 0,33768637 eV (dixième d'eV)

In [ ]:
data = bsplotter.bs_plot_data()
pl = bsplotter.get_plot()
E = data.get('energy').get('1')
D = data.get('ticks').get('distance')

vbm = band.get_vbm()
##print("Index vbm :" , vbm['band_index'])   
cbm = band.get_cbm()
##print("Index cbm :" ,cbm['band_index'])

Vindex = 16 
Cindex = 17
#les indices max trouvés

slopeV = np.zeros(10)
slopeC = np.zeros(10)
Dbranch = np.unique(D)
for i in range(10):
    DeltaD = Dbranch[i+1]-Dbranch[i]
    slopeV[i] = abs(((E[i][Vindex])[-1] - (E[i][Vindex])[0])/(DeltaD) )
    slopeC[i] = abs(((E[i][Cindex])[-1] - (E[i][Cindex])[0])/(DeltaD) )

#print ("la liste slopeV ", slopeV)
#print ("la liste slopeC ", slopeC)
    
maxV = np.max(slopeV) #indice 1
minV = np.min(slopeV) # indice 5 & 8


print("La dispersion maximale dans la dernière bande (valence) vaut ",maxV," (en bleu sur le graph) \n" )
print("La dispersion minimale dans la dernière bande (valence) vaut ",minV," (en rouge sur le graph) \n" )

maxC = np.max(slopeC) #indice 1
minC = np.min(slopeC) #indice 4
print("La dispersion maximale dans la première bande (conduction) vaut ",maxV," (en vert sur le graph) \n" )
print("La dispersion minimale dans la première bande (condution) vaut ",maxV," (en vert sur le graph) \n" )


plt.arrow(Dbranch[1], (E[1][Cindex])[0], Dbranch[2] - Dbranch[1], (E[1][Cindex])[-1] - (E[1][Cindex])[0], head_width=0.15, head_length=0.05, linewidth=3, color='g', length_includes_head=True)
plt.arrow(Dbranch[4], (E[4][Cindex])[0], Dbranch[5] - Dbranch[4], (E[4][Cindex])[-1] - (E[4][Cindex])[0], head_width=0.15, head_length=0.05, linewidth=3, color='y', length_includes_head=True)


plt.arrow(Dbranch[1], (E[1][Vindex])[0], Dbranch[2] -Dbranch[1], (E[1][Vindex])[-1] - (E[1][Vindex])[0], head_width=0.15, head_length=0.05, linewidth=3, color='b', length_includes_head=True)

plt.arrow(Dbranch[5], (E[5][Vindex])[0], Dbranch[6] -Dbranch[5], (E[5][Vindex])[-1] - (E[5][Vindex])[0], head_width=0.15, head_length=0.05, linewidth=3, color='r', length_includes_head=True)
plt.arrow(Dbranch[8], (E[8][Vindex])[0], Dbranch[9] -Dbranch[8], (E[8][Vindex])[-1] - (E[8][Vindex])[0], head_width=0.15, head_length=0.05, linewidth=3, color='r', length_includes_head=True)


plt.show()

# Calcul de la masse effective


In [ ]:
bsplotter.get_plot(vbm_cbm_marker = True) #max et min

h_bar = 6.5821e-16 # en eV

val = bandstruct.get_vbm()
E_val = val['energy']
kpoint_val = val['kpoint'].cart_coords

mEffVal = (h_bar**2 * (kpoint_val[0]**2+kpoint_val[1]**2+kpoint_val[2]**2))/(2*abs(E_val))

print("La masse effective au sommet de la dernière bande de valence (dispersion parabolique) = " + 
      str(mEffVal)+ "[kg]\n")

cond = band.get_cbm()
E_cond = cond['energy']
kpoint_cond = cond['kpoint'].cart_coords

mEffCond = (h_bar**2 * (kpoint_cond[0]**2+kpoint_cond[1]**2+kpoint_cond[2]**2))/(2*abs(E_cond))

print("La masse effective à la base de la première bande de conduction (dispersion parabolique) = " + 
      str(mEffCond) + "[kg]")



plt.show()